In [2]:
from sklearn import metrics
import pandas as pd
from torch import nn
from pyarabic.araby import tokenize
import numpy as np
import pickle
import spacy
import torch

# from model_building import Classifier 
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector
from gru_model import ArabicDataset, Classifier, evaluate, train

In [3]:
# needed functions
def print_report(y_pred, y_test):
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [4]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

c:\Users\Nada\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'ar_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))

In [8]:
# load word2index dictionary
with open('./vocab/stance/word2index.pickle', 'rb') as f:
    word2index = pickle.load(f)

In [9]:
weights_train_matrix = []
for word in word2index:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

td-idf feature

In [11]:
# load naive bayes model
with open('./models/stance/NaiveBayes_tfidf.sav', 'rb') as f:
    naive_bayes_model = pickle.load(f)

with open('./models/stance/TFIDFVectorizer.sav', 'rb') as f:
    word_vectorizer = pickle.load(f)

X_test_tfidf = word_vectorizer.transform(test_data['text'])
y_pred = naive_bayes_model.predict_proba(X_test_tfidf)

Ara2Vec embedding

In [12]:
test_data_embeddings = np.array([np.array([nlp(i).vector for i in ls]) for ls in test_data["text"]], dtype=object)
_, X_test_vect_avg = avg_word_vector([], test_data_embeddings)

# load SVC model
with open('./models/stance/RandomForest_Ara2Vec.sav', 'rb') as f:
    svc_model = pickle.load(f)

y_pred += svc_model.predict_proba(X_test_vect_avg)

In [14]:
#load GRU model
with open('./models/stance/GRU_Ara2Vec.pth', 'rb') as f:
    gru_model = Classifier(weights_train_matrix)
    gru_model.load_state_dict(torch.load(f)) 
    gru_model.eval()

test_data_tokenized = test_data['text'].apply(tokenize)
test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['stance'] + 1, word2index['<فراغ>'])
y_pred += evaluate(gru_model,test_dataset)

100%|██████████| 32/32 [00:01<00:00, 29.16it/s]


accuracy: 0.730
              precision    recall  f1-score   support

           0       0.32      0.27      0.29        70
           1       0.30      0.60      0.40       126
           2       0.92      0.79      0.85       804

    accuracy                           0.73      1000
   macro avg       0.51      0.55      0.51      1000
weighted avg       0.80      0.73      0.76      1000


Test Accuracy: 0.7300000190734863


In [15]:
y_pred /= 3
y_pred = np.argmax(y_pred, axis=1) - 1
print_report(y_pred, test_data['stance'])

              precision    recall  f1-score   support

          -1       0.32      0.26      0.29        70
           0       0.33      0.57      0.42       126
           1       0.92      0.83      0.87       804

    accuracy                           0.75      1000
   macro avg       0.52      0.55      0.52      1000
weighted avg       0.80      0.75      0.77      1000

accuracy: 0.754
